##Version Date: October 13 2023, Henrik Loecke

In [1]:
#PERMANENT CELL 1

import os
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import math
import pandas as pd
import numpy as np
import datetime as dt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ctypes
import traceback
MessageBox = ctypes.windll.user32.MessageBoxA
from Result_Lookup_Variables import *
import subprocess

In [2]:
class Node():
    def __init__(self,node):    
        self.node = node
    def invert(self):        
        return node_df.loc[self.node,'Invert Level']
    def ground(self):        
        return node_df.loc[self.node,'Ground Level']
    def soh(self):        
        return node_df.loc[self.node,'Safe Operating Head']
    def asset_name(self):        
        return node_df.loc[self.node,'Asset Name']
    def cover(self):        
        return node_df.loc[self.node,'Cover Type']

class Pipe():
    def __init__(self,pipe):    
        self.pipe = pipe
    def height(self):        
        return pipe_df.loc[self.pipe,'Height'] 
    def uplevel(self):
        uplevel = pipe_df.loc[self.pipe,'UpLevel']
        if math.isnan(uplevel):
            node = pipe_df.loc[self.pipe,'FromNode']
            uplevel = node_df.loc[node,'Invert Level']
        return uplevel
    def dwlevel(self):
        dwlevel = pipe_df.loc[self.pipe,'DwLevel']
        if math.isnan(dwlevel):
            node = pipe_df.loc[self.pipe,'ToNode']
            dwlevel = node_df.loc[node,'Invert Level']
        return dwlevel
    

In [ ]:
for m in master_list:
    
    [model_area,model,result_folder,output_folder,res_list,groupby_acronym_owner,pipe_ls_exclusions]
    
    model_area = m[0]
    model = m[1]
    result_folder = m[2]  
    output_folder = m[3]
    result_list = m[4]
    groupby_acronym_owner = m[5]
    pipe_ls_exclusions = m[6]
    name_shortenings = m[8]
    
    #Prepare button input
    buttons_ons = []
    result_groups = []
    result_groups.extend(result[2] for result in result_list if result[2] not in result_groups)
    button_ons_all = [True,True,True,True]
    for i, result_group in enumerate(result_groups):
        button_ons = [True,True,True,True]
        for result in result_list:
            if i == 0: 
                button_ons_all.append(True)
            if result[2] == result_group:
                button_ons.append(True)
            else:
                button_ons.append(False)
        if i == 0:
            buttons_ons.append(['All',button_ons_all])
        button_text = result_group
        for name_shortening in name_shortenings:
            button_text = button_text.replace(name_shortening[0],name_shortening[1])
        buttons_ons.append([button_text,button_ons])
        
    #Read model data
    parameter_script = r"Acronym_Crawler.py"
    bat_file_path = 'Read_Parameters.bat'
    bat_file = open(bat_file_path, "w")
    model_path = result_folder + '\\' + model
    bat_file.write('"' + parameter_script + '" "' + os.getcwd() + '" "' + model_path + '"')
    bat_file.close()
    result = subprocess.call([bat_file_path]) 
    if os.path.exists(model_path) == False:
        raise ValueError("The variable 'model_path' points to a path that does not exist: " + model_path)
    if result == 1: #Error
        raise ValueError("The sub process threw an error. Please Locate the bat file: " + bat_file_path + ", open it in notepad, \
        then add a new line and type in letters only: Pause. Double click the bat file to run it and it will show the error.")
       
    node_df = pd.read_csv(os.getcwd() + '\\ls_nodes.csv',dtype={'Node': str})
    node_df.set_index('Node',inplace=True)
    node_df['Asset Name'].fillna('',inplace=True)
    pipe_df = pd.read_csv(os.getcwd() + '\\ls_pipes.csv',dtype={'Pipe': str,'FromNode': str,'ToNode': str})
    pipe_df.set_index('Pipe',inplace=True)
    for pipe in pipe_ls_exclusions:
        pipe_df.drop(pipe,inplace=True)
  
    print('Loading results in ' + result_folder)
    for r in result_list:
        header = r[0]
        file = r[1]
        result_subfolder = ''
        #find subfolders for MIKE+
        if model[-7:] == '.sqlite':
            file_found = False
            for f1 in os.listdir(result_folder):
                if f1[-7:] == '.sqlite':
                    #browse subfolder              
                    try:
                        check_subfolder = os.path.basename(f1)[:-7] + '_m1d - Result Files'
                        for f2 in os.listdir(result_folder + '\\' + check_subfolder):
                            if os.path.basename(f2) == file:
                                result_subfolder = '\\' + check_subfolder
                                file_found = True
                    except:
                        pass
            if not file_found:
                message = 'The following result file was not found:\n' + file
                raise ValueError(message) 
                     
        res1d = Res1D(result_folder + result_subfolder + '\\' + file)
        print('Loading ' + file)
        r.append(res1d)
        
        
    missing_pipes = []

    node_delta_xs = [['US',-1],['DS',0]]

    acronyms = list(pipe_df.Acronym.unique())
    acronyms.sort()
    paths = []

    loop_counter = 0
    
    print ("Building path")

    for j, acronym in enumerate(acronyms):
        print ("Building path for " + acronym + " number " + str(j+1) + " of " + str(len(acronyms)) + " at " + str(dt.datetime.now()))
        
        path_no = 0

        pipe_index_df = pipe_df[pipe_df.Acronym==acronym].copy()

        while len(pipe_index_df)>0:


            start_pipes = []
            for index, row in pipe_index_df.iterrows():
                if row['FromNode'] not in list(pipe_index_df.ToNode):

                    start_pipes.append(index)

            start_pipes

            for start_pipe in start_pipes[:1]:
                exit_path = False
                chainage = 0

                node = pipe_index_df.loc[start_pipe,'FromNode']
                pipe = start_pipe

                path = []

                for node_delta_x in node_delta_xs:
                    point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                             Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                            np.nan,np.nan]
                    for r in result_list:
                        values = r[3].query.GetNodeValues(node, "WaterLevel")
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan)                    
                    path.append(point)

                i = 0
                while exit_path == False and i < 1000:

                    point = [path_no,i*3+1,acronym,'Pipe US',pipe,chainage,Pipe(pipe).uplevel(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).uplevel()]

                    for r in result_list:                    
                        try:
                            values = r[3].query.GetReachStartValues(pipe, "WaterLevel")
                        except:
                            values = None
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan) 

                    path.append(point)

                    chainage += pipe_index_df.loc[pipe,'Length']

                    node = pipe_index_df.loc[pipe,'ToNode']


                    point = [path_no,i*3+1,acronym,'Pipe DS',pipe,chainage-1,Pipe(pipe).dwlevel(),
                                 Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                Pipe(pipe).height(),Pipe(pipe).height()+Pipe(pipe).dwlevel()]

                    for r in result_list:
                        try:
                            values = r[3].query.GetReachEndValues(pipe, "WaterLevel")
                        except:
                            values = None
                        if values != None:
                            point.append(max(values))
                        else:
                            point.append(np.nan) 

                    path.append(point)


                    for node_delta_x in node_delta_xs:
                        point = [path_no,0,acronym,'Node-'+node_delta_x[0],node,chainage+node_delta_x[1],Node(node).invert(),
                                     Node(node).ground(),Node(node).soh(),Node(node).asset_name(),Node(node).cover(),
                                    np.nan,np.nan]

                        for r in result_list:
                            values = r[3].query.GetNodeValues(node, "WaterLevel")
                            if values != None:
                                point.append(max(values))
                            else:
                                point.append(np.nan)                 
                        path.append(point)

                    pipe_index_df.drop(pipe,inplace=True)

                    try:
                        pipe = pipe_index_df[pipe_index_df.FromNode==node].index[0]
                    except:
                        exit_path = True   
                    i += 1

                    loop_counter += 1
                    if loop_counter > 20000:
                        raise


            paths += path

            path_no += 1

    # paths
    columns = ['Path Number','Sqn','Acronym','Type','MUID','Chainage','Invert Level','Ground Level','Safe Operating Head',
                                       'Asset Name','Cover Type','Height','Pipe Obvert']

    for r in result_list:
        header = r[0]
        columns.append(header)


    path_df=pd.DataFrame(paths,columns=columns)
    path_df.replace(999,np.nan,inplace=True)
    
    # output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Always_Latest_Master_Model_Simulations_FSA\Plots'

    colors = ['black','black','green','red','brown','orange','grey','brown','midnightblue','tan','firebrick','darkturquoise','darkorchid','dimgray','darkgoldenrod','indianred','deeppink','AliceBlue','AntiqueWhite','Aqua','Aquamarine','Azure','Beige','Bisque','Black','BlanchedAlmond','Blue','BlueViolet','Brown','BurlyWood','CadetBlue','Chartreuse','Chocolate','Coral','CornflowerBlue','Cornsilk','Crimson','Cyan','DarkBlue','DarkCyan','DarkGoldenrod','DarkGray','DarkGreen','DarkGrey','DarkKhaki','DarkMagenta','DarkOliveGreen','DarkOrange','DarkOrchid','DarkRed','DarkSalmon','DarkSeaGreen','DarkSlateBlue','DarkSlateGray','DarkSlateGrey','DarkTurquoise','DarkViolet','DeepPink','DeepSkyBlue','DimGray','DodgerBlue','FireBrick','FloralWhite','ForestGreen','Fuchsia','Gainsboro','GhostWhite','Gold','Goldenrod','Gray','Green','GreenYellow','Grey','Honeydew','HotPink','IndianRed','Indigo','Ivory','Khaki','Lavender','LavenderBlush','LawnGreen','LemonChiffon','LightBlue','LightCoral','LightCyan','LightGoldenrodYellow','LightGray','LightGreen','LightGrey','LightPink','LightSalmon','LightSeaGreen','LightSkyBlue','LightSlateGray','LightSlateGrey','LightSteelBlue','LightYellow','Lime','LimeGreen','Linen','Magenta','Maroon','MediumAquamarine','MediumBlue','MediumOrchid','MediumPurple','MediumSeaGreen','MediumSlateBlue','MediumSpringGreen','MediumTurquoise','MediumVioletRed','MidnightBlue','MintCream','MistyRose','Moccasin','NavajoWhite','Navy','OldLace','Olive','OliveDrab','Orange','OrangeRed','Orchid','PaleGoldenrod','PaleGreen','PaleTurquoise','PaleVioletRed','PapayaWhip','PeachPuff','Peru','Pink','Plum','PowderBlue','Purple','Rebeccapurple','Red','RosyBrown','RoyalBlue','SaddleBrown','Salmon','SandyBrown','SeaGreen','Seashell','Sienna','Silver','SkyBlue','SlateBlue','SlateGray','SlateGrey','Snow','SpringGreen','SteelBlue','Tan','Teal','Thistle','Tomato','Turquoise','Violet','Wheat','White','WhiteSmoke','Yellow','YellowGreen']

    print ("Writing htmls under " + output_folder)
    for j, acronym in enumerate(acronyms):

        print ("Writing html for " + acronym + " number " + str(j+1) + " of " + str(len(acronyms)) + " at " + str(dt.datetime.now()))

        path_acronym_df = path_df[path_df.Acronym==acronym]

        output_subfolders = [output_folder + "\\All_Longsections"]
        if groupby_acronym_owner == True:
            output_subfolders.append(output_folder + '\\GV_Acronym_' + acronym)

        for i, output_subfolder in enumerate(output_subfolders):

            relative_output_subfolder = '..\\' + os.path.basename(output_subfolder)

            if not os.path.isdir(output_subfolder): os.makedirs(output_subfolder) 

            with open(output_subfolder + "\\Long_Section_" + acronym + ".html", 'w') as f:
                f.write('<link rel="stylesheet" href="..\Maps_And_CSS\style.css">\n')

                f.write('<!DOCTYPE html>\n')
                f.write('<html>\n')
                f.write('<head>\n')
                f.write('<meta charset="utf-8">\n')
                f.write('</head>\n')
                f.write('<body>\n')
                f.write('<div class="sidenav">\n')


                f.write("<h2>Links to Others (Alphabetic Order)</h2>\n")
                if j == 0:
                    previous_acronym = acronyms[len(acronyms)-1]
                else:
                    previous_acronym = acronyms[j-1]
                if j == len(acronyms)-1:
                    next_acronym = acronyms[0]
                else:
                    next_acronym = next_acronym = acronyms[j+1]

                if i == 0:
                    path = relative_output_subfolder +  "\\Long_Section_" + previous_acronym + ".html"
                else:                
                    path = '..\\GV_Acronym_' + previous_acronym + "\\Long_Section_" + previous_acronym + ".html"

                f.write('<a href="' + path + '">Previous (' + previous_acronym + ')</a>\n')
                f.write("<br>")

                if i == 0:
                    path = relative_output_subfolder +  "\\Long_Section_" + next_acronym + ".html"
                else:                
                    path = '..\\GV_Acronym_' + next_acronym + "\\Long_Section_" + next_acronym + ".html"

                f.write('<a href="' + path + '">Next (' + next_acronym + ')</a>\n')
                f.write("<br>")            

                f.write("<h2>Map</h2>\n")

                map_string = '<img src="..\\Maps_And_CSS\\Acronym_' + acronym + '.jpg" alt="' + acronym + '">\n'
                f.write(map_string + "\n")
                f.write("<p></p>\n")

                f.write('</div>')
                f.write('<div class="main">\n')
                f.write("<h1>Long Sections for " + acronym + "</h1>\n")

                for path_no in path_acronym_df['Path Number'].unique():
                    path_acronym_sub_df = path_acronym_df[path_acronym_df['Path Number']==path_no]
                    path_acronym_node_df = path_acronym_sub_df[path_acronym_sub_df.Type=='Node-DS']

                    fig = go.Figure()

                    columns=['Invert Level','Pipe Obvert','Ground Level','Safe Operating Head']
                    for r in result_list:
                        header = r[0]
                        columns.append(header)

                    for i, column in enumerate(columns):
                        if i < 4:
                            line_width = 4
                        else:
                            line_width = 2
                        fig.add_trace(go.Scatter(x=path_acronym_sub_df.Chainage, 
                                                 y = path_acronym_sub_df[column],line_color=colors[i],name=column, line_width = line_width))

                    lbl_x = []
                    lbl_y = []
                    lbl_text = []

                    label_y = max(path_acronym_node_df['Ground Level']) + 2

                    for index, row in path_acronym_node_df.iterrows():

                        fig.add_shape(type="rect",x0=row['Chainage']-1, y0=row['Invert Level'], x1=row['Chainage'],
                                      y1=row['Ground Level'],line=dict(color="RoyalBlue",width=1),)

                        fig.add_shape(type="line",
                            x0=row['Chainage']-0.5, y0=row['Ground Level'], x1=row['Chainage']-0.5, y1=label_y-0.8,
                            line=dict(
                                color="Grey",
                                width=1,
                                dash="dot",
                            )
                        )

                        lbl_x.append(row['Chainage'] - 0.5)

                        lbl_y.append(label_y)
                        if row['Cover Type']==2:
                            cover_type = 'Sealed'
                        else:
                            cover_type = 'Unsealed'
                        lbl_text.append(str(row['Asset Name']) + '<br>' +  str(row['MUID']) + '<br>' + cover_type)

                    fig.add_trace(go.Scatter(
                        x=lbl_x,
                        y=lbl_y,
                        text=lbl_text,
                        mode="text",
                        name="Labels"
                    ))

                    fig.update_layout(
                        autosize=False,
                        width = 1362,
                        height=700,
                        margin=dict(
                            l=50,
                            r=50,
                            b=25,
                            t=35,
                            pad=4
                            ),
                        yaxis_title='Geodetic Elevation(m)',
                        xaxis_title='Chainage (m)',
                        

                        updatemenus=[
                            {
                                'buttons': [
                                    {
                                        'args': [{'visible': button_on[1]}, {'title': button_on[0]}],
                                        'label': button_on[0],
                                        'method': 'update'
                                    }
                                    for button_on in buttons_ons
                                ],
                                'direction': 'left',
                                'pad': {'r': 10, 't': 87},
                                'showactive': True,
                                'type': 'buttons',
                                'x': 0.1,
                                'xanchor': 'left',
                                'y': 0.06,
                                'yanchor': 'top'
                            }
                            ]                                           
                        )

                    f.write("<h2>Long Section " + str(path_no+1) + "</h2>\n")
                    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

                f.write('</div>')
                f.write('</body>')
                f.write('</html>')
                f.close()

Loading results in J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\X_Times_BSF\EFLSP_Modelling\Phase_0\Model_V187_Ann0p8_0p3cms
Loading FSA_BSF_16p8k_2030pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_15p68k_2035pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_15p12k_2040pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_14p56k_2045pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_14k_2050pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_12p88k_2060pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_11p2k_2075pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_11p2k_2090pop_S_V_2030_NetworkDefault_Network_HD.res1d
Loading FSA_BSF_11p2k_2100pop_S_V_2030_NetworkDefault_Network_HD.res1d
Building path
Building path for ALT number 1 of 207 at 2024-12-20 12:34:10.861154
Building path for ALX number 2 of 207 at 2024-12-20 12:34:11.559556
Building path for BBA number 3 of 207 at 2024-12-20 12:34:12.911077
Building path for

Building path for NSX number 109 of 207 at 2024-12-20 12:36:03.230886
Building path for NTS number 110 of 207 at 2024-12-20 12:36:03.822356
Building path for NWA number 111 of 207 at 2024-12-20 12:36:04.719686
Building path for NWC number 112 of 207 at 2024-12-20 12:36:05.735996
Building path for NWD number 113 of 207 at 2024-12-20 12:36:07.837747
Building path for NWF number 114 of 207 at 2024-12-20 12:36:08.578128
Building path for NWG number 115 of 207 at 2024-12-20 12:36:10.029721
Building path for NWP number 116 of 207 at 2024-12-20 12:36:10.650108
Building path for NWR number 117 of 207 at 2024-12-20 12:36:11.117761
Building path for NWS number 118 of 207 at 2024-12-20 12:36:12.637286
Building path for NWT number 119 of 207 at 2024-12-20 12:36:13.865989
Building path for NWW number 120 of 207 at 2024-12-20 12:36:14.404856
Building path for NWX number 121 of 207 at 2024-12-20 12:36:15.898903
Building path for NXA number 122 of 207 at 2024-12-20 12:36:16.699377
Building path for NX

Writing html for BYN number 20 of 207 at 2024-12-20 12:37:30.447140
Writing html for C2F number 21 of 207 at 2024-12-20 12:37:30.533665
Writing html for CB1 number 22 of 207 at 2024-12-20 12:37:30.747613
Writing html for CFF number 23 of 207 at 2024-12-20 12:37:38.450444
Writing html for CI2 number 24 of 207 at 2024-12-20 12:37:39.147816
Writing html for CIN number 25 of 207 at 2024-12-20 12:37:40.051492
Writing html for CMP number 26 of 207 at 2024-12-20 12:37:40.324166
Writing html for CQC number 27 of 207 at 2024-12-20 12:37:40.371713
Writing html for CQD number 28 of 207 at 2024-12-20 12:37:40.840395
Writing html for CQL number 29 of 207 at 2024-12-20 12:37:43.526455
Writing html for CQM number 30 of 207 at 2024-12-20 12:37:44.067773
Writing html for CTO number 31 of 207 at 2024-12-20 12:37:44.490332
Writing html for CTS number 32 of 207 at 2024-12-20 12:37:44.535373
Writing html for CV6 number 33 of 207 at 2024-12-20 12:37:44.902164
Writing html for CV7 number 34 of 207 at 2024-12

Writing html for OPA number 142 of 207 at 2024-12-20 12:38:34.619193
Writing html for OPC number 143 of 207 at 2024-12-20 12:38:34.726866
Writing html for OSR number 144 of 207 at 2024-12-20 12:38:37.242664
Writing html for P2F number 145 of 207 at 2024-12-20 12:38:38.111845
Writing html for PFM number 146 of 207 at 2024-12-20 12:38:44.467773
Writing html for PIF number 147 of 207 at 2024-12-20 12:38:45.404667
Writing html for PIM number 148 of 207 at 2024-12-20 12:38:45.788894
Writing html for PMA number 149 of 207 at 2024-12-20 12:38:46.421296
Writing html for POC number 150 of 207 at 2024-12-20 12:38:46.542946
Writing html for PQE number 151 of 207 at 2024-12-20 12:38:47.172423
Writing html for PQO number 152 of 207 at 2024-12-20 12:38:47.316876
Writing html for PSE number 153 of 207 at 2024-12-20 12:38:47.391297
Writing html for PSM number 154 of 207 at 2024-12-20 12:38:47.627193
Writing html for PSO number 155 of 207 at 2024-12-20 12:38:48.502180
Writing html for PSS number 156 of

Building path for GLE number 43 of 207 at 2024-12-20 12:43:19.266043
Building path for GLK number 44 of 207 at 2024-12-20 12:43:21.018654
Building path for GLN number 45 of 207 at 2024-12-20 12:43:21.591364
Building path for GLO number 46 of 207 at 2024-12-20 12:43:22.030802
Building path for GLT number 47 of 207 at 2024-12-20 12:43:22.630538
Building path for HTA number 48 of 207 at 2024-12-20 12:43:22.890409
Building path for HTB number 49 of 207 at 2024-12-20 12:43:24.016257
Building path for KTZ number 50 of 207 at 2024-12-20 12:43:24.222895
Building path for KYT number 51 of 207 at 2024-12-20 12:43:24.292173
Building path for KZS number 52 of 207 at 2024-12-20 12:43:25.199827
Building path for L1F number 53 of 207 at 2024-12-20 12:43:25.503829
Building path for L2F number 54 of 207 at 2024-12-20 12:43:26.276055
Building path for L3C number 55 of 207 at 2024-12-20 12:43:27.491720
Building path for LAF number 56 of 207 at 2024-12-20 12:43:28.583554
Building path for LAG number 57 of

...

Writing html for MLT number 73 of 207 at 2024-12-20 12:46:39.765251
Writing html for MPA number 74 of 207 at 2024-12-20 12:46:40.979100
Writing html for MPK number 75 of 207 at 2024-12-20 12:46:41.422649
Writing html for MPR number 76 of 207 at 2024-12-20 12:46:45.159194
Writing html for MPY number 77 of 207 at 2024-12-20 12:46:46.081368
Writing html for MRS number 78 of 207 at 2024-12-20 12:46:46.489040
Writing html for MS2W number 79 of 207 at 2024-12-20 12:46:46.574051
Writing html for MSO number 80 of 207 at 2024-12-20 12:46:46.784308
Writing html for MST number 81 of 207 at 2024-12-20 12:46:46.841679
Writing html for MSW number 82 of 207 at 2024-12-20 12:46:47.014345
Writing html for N2T number 83 of 207 at 2024-12-20 12:46:47.062546
Writing html for NCT number 84 of 207 at 2024-12-20 12:46:47.801037
Writing html for NDI number 85 of 207 at 2024-12-20 12:46:49.047086
Writing html for NDR number 86 of 207 at 2024-12-20 12:46:50.999426
Writing html for NET number 87 of 207 at 2024-1

Writing html for SST number 193 of 207 at 2024-12-20 12:47:44.675767
Writing html for STF number 194 of 207 at 2024-12-20 12:47:45.297489
Writing html for SYC number 195 of 207 at 2024-12-20 12:47:45.542824
Writing html for SYC2 number 196 of 207 at 2024-12-20 12:47:47.683178
Writing html for TSF number 197 of 207 at 2024-12-20 12:47:48.413015
Writing html for TSI number 198 of 207 at 2024-12-20 12:47:48.590255
Writing html for VIN number 199 of 207 at 2024-12-20 12:47:48.644756
Writing html for VT2F number 200 of 207 at 2024-12-20 12:47:48.881684
Writing html for VTH number 201 of 207 at 2024-12-20 12:47:49.249710
Writing html for WDO number 202 of 207 at 2024-12-20 12:47:49.477643
Writing html for WDT number 203 of 207 at 2024-12-20 12:47:49.635630
Writing html for WFM number 204 of 207 at 2024-12-20 12:47:50.040331
Writing html for WHR number 205 of 207 at 2024-12-20 12:47:51.783806
Writing html for WMT number 206 of 207 at 2024-12-20 12:47:51.847369
Writing html for WMW number 207 

Building path for NPL number 94 of 207 at 2024-12-20 12:53:48.807160
Building path for NR2L number 95 of 207 at 2024-12-20 12:53:49.169625
Building path for NRT number 96 of 207 at 2024-12-20 12:53:51.479508
Building path for NS4 number 97 of 207 at 2024-12-20 12:53:52.108259
Building path for NS4E1 number 98 of 207 at 2024-12-20 12:53:52.619161
Building path for NS6 number 99 of 207 at 2024-12-20 12:53:53.032449
Building path for NSA number 100 of 207 at 2024-12-20 12:53:53.812996
Building path for NSD number 101 of 207 at 2024-12-20 12:53:54.173105
Building path for NSE number 102 of 207 at 2024-12-20 12:53:55.258679
Building path for NSF number 103 of 207 at 2024-12-20 12:53:55.731701
Building path for NSM number 104 of 207 at 2024-12-20 12:53:57.425898
Building path for NSP number 105 of 207 at 2024-12-20 12:53:59.525202
Building path for NSR number 106 of 207 at 2024-12-20 12:54:01.738636
Building path for NSS number 107 of 207 at 2024-12-20 12:54:03.904830
Building path for NSW n

Writing html for BBA number 3 of 207 at 2024-12-20 12:55:27.067234
Writing html for BBB number 4 of 207 at 2024-12-20 12:55:27.598420
Writing html for BBC number 5 of 207 at 2024-12-20 12:55:28.450915
Writing html for BBL number 6 of 207 at 2024-12-20 12:55:28.662155
Writing html for BBX number 7 of 207 at 2024-12-20 12:55:29.670849
Writing html for BGO number 8 of 207 at 2024-12-20 12:55:29.883560
Writing html for BIN number 9 of 207 at 2024-12-20 12:55:29.947910
Writing html for BLNI number 10 of 207 at 2024-12-20 12:55:31.459832
Writing html for BNC number 11 of 207 at 2024-12-20 12:55:32.557455
Writing html for BNL number 12 of 207 at 2024-12-20 12:55:32.870400
Writing html for BNP number 13 of 207 at 2024-12-20 12:55:33.185018
Writing html for BNS number 14 of 207 at 2024-12-20 12:55:33.310173
Writing html for BSE number 15 of 207 at 2024-12-20 12:55:33.764604
Writing html for BSI number 16 of 207 at 2024-12-20 12:55:33.918643
Writing html for BST number 17 of 207 at 2024-12-20 12

Writing html for NXG number 127 of 207 at 2024-12-20 12:56:41.802202
Writing html for NXI number 128 of 207 at 2024-12-20 12:56:41.848507
Writing html for NXJ number 129 of 207 at 2024-12-20 12:56:41.907972
Writing html for NXK number 130 of 207 at 2024-12-20 12:56:41.956433
Writing html for NXP number 131 of 207 at 2024-12-20 12:56:42.082816
Writing html for NZA number 132 of 207 at 2024-12-20 12:56:42.129058
Writing html for NZB number 133 of 207 at 2024-12-20 12:56:42.223682
Writing html for NZC number 134 of 207 at 2024-12-20 12:56:42.351532
Writing html for NZD number 135 of 207 at 2024-12-20 12:56:42.510654
Writing html for NZE number 136 of 207 at 2024-12-20 12:56:42.625549
Writing html for NZF number 137 of 207 at 2024-12-20 12:56:42.752702
Writing html for NZG number 138 of 207 at 2024-12-20 12:56:42.810156
Writing html for NZH number 139 of 207 at 2024-12-20 12:56:42.882893
Writing html for NZI number 140 of 207 at 2024-12-20 12:56:42.933220
Writing html for OP6 number 141 of